In [ ]:
import torch
import numpy as np
from diff_trainer import extract
from matplotlib import pyplot as plt
import diffusers
import os
from fucs import save_image
from coders import cal_acc

In [ ]:
import Unetwork
import diff_trainer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
modelconfig = {
    'device': device,
    'batch_size': 32,
    'T': 100,
    'channel': 8,
    'channel_mult': [2,4,4,2],
    'atten': [1,3],
    'num_res_block': 3,                  #不知道是多少，随便写一个吧
    'dropout': 0.2,                      #dropout置零率，在0到1之间
    'training_load_weight': None,        #填从那个文件夹里加载权重
    'eval_load_weight': 'ckpt5_.pt',  #填从那个文件夹里加载权重
    'save_weight_dir': 'E:/D1/diffusion/my_models/my_model_data/train_weight3/',#把权重保存在那个文件夹里
    'picture_save_path': 'E:/D1/diffusion/my_models/my_model_data/picture/',#把生成的图片保存在那个文件夹里
    'lr': 0.001,
    'epoch': 1001,                        #不知道是多少，x修改了这个注释
    'multiplier': 1.0,                   #不知道填啥
    'bata_1':0.0001,
    'bata_T':0.02,
    'grad_clip': 2.0,
}

In [ ]:
with torch.no_grad():
    device = torch.device(modelconfig['device'])
    model = Unetwork.UNet(
        T=modelconfig["T"],
        in_ch=5,
        ch=modelconfig["channel"],
        ch_mult=modelconfig["channel_mult"],
        atten=modelconfig["atten"],
        num_res_block=modelconfig["num_res_block"],
        dropout=modelconfig["dropout"]
    )
    ckpt = torch.load(
        os.path.join(modelconfig['save_weight_dir'], modelconfig['eval_load_weight']),
        map_location=device
    )
    model.load_state_dict(ckpt)
    model.eval()


In [ ]:
x0 = np.load('E:/D1/diffusion/my_models/my_model_data/picture/real_target2.npy')
noise = np.load('E:/D1/diffusion/my_models/my_model_data/picture/data_noise2.npy')
condition = np.load('E:/D1/diffusion/my_models/my_model_data/picture/condition2.npy')

x0 = torch.tensor(x0).unsqueeze(0).unsqueeze(0)
noise = torch.tensor(noise).unsqueeze(0).unsqueeze(0)
condition = torch.tensor(condition).unsqueeze(0)
# save_image(x0, names=['x0'], show=True)
# save_image(x0, noise, names=['x0', 'noise'], show=True)
save_image(x0, noise, condition, names=['x0', 'noise', 'condition'], show=True)

# noise_to_add = torch.randn_like(x0)
noise_to_add = noise

In [ ]:
ddpm_sch = diffusers.schedulers.scheduling_ddpm.DDPMScheduler(num_train_timesteps=100, beta_schedule='squaredcos_cap_v2',)
for t in reversed(range(100)):
    print(t)
    xt = ddpm_sch.add_noise(x0, noise_to_add, torch.tensor([t]))
    save_image(xt, noise_to_add, names=['x'+str(t), 'noise_to_add'], show=True)
    


In [ ]:
xt = noise
# save_image(xt, 'xt', show=True)
for t in reversed(range(100)):
    eps = model(torch.cat((xt, condition), dim=1), torch.tensor([t]))
    eps2 = (eps - eps.mean()) / eps.std()
    save_image(xt, eps2, names=['x'+str(t), 'eps'+str(t)], show=True)
    xt_1 = ddpm_sch.step(eps2, torch.tensor([t]), xt)
    xt = xt_1[0]
    # if t % 10 == 0:
    #     save_image(xt, eps2, names=['x'+str(t), 'eps'+str(t)], show=True)